<a href="https://colab.research.google.com/github/kmalicekim/NLP_practice_mine/blob/master/7_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 7.2 사이킷런을 미용한 토픽 모델링

### 7.2.1 데이터 준비

In [10]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train= fetch_20newsgroups(subset='train')
newsgroups_train.target_names


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [16]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space', 
              'comp.sys.ibm.pc.hardware', 'sci.crypt', 'rec.sport.hockey']

# 학습 데이터셋을 가져옴
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Selected categories:', newsgroups_train.target_names)

#Train set size: 3819
#Selected categories: ['alt.atheism', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'sci.crypt', 'sci.space', 'talk.religion.misc']


In [18]:
newsgroups_train.data[:5]

["From: michael@jester.GUN.de (Michael Gerhards)\nDistribution: world\nSubject: Re: HELP: my pc freezes!\nX-Newsreader: TIN [version 1.1 PL8]\nOrganization: private COHERENT system\nLines: 15\n\nPerry Egelmeers (perry@wswiop11.win.tue.nl) wrote:\n> ladanyi@cs.cornell.edu (La'szlo' Lada'nyi) writes:\n\n> >Problem: Occasionaly the machine freezes. At least that's what I thought, but\n> >recently I discovered that the machine works, just the keyboard freezes and\n> >the clock drops down from turbo (33Mhz) to standard (16Mhz) mode.\n\n> Perhaps you hit the ^S (Control S)?  Try ^Q.\n> I know it doesn't explain the clock rate drop...\n\nWe had the same problem in our company. We changed the keyboard-bios and\nafter that, everything went fine. Our dealer told us that some boards of\nthat series have a defect kbd-bios.\n\nMichael\n--\n*  michael@jester.gun.de  *   Michael Gerhards   *   Preussenstrasse 59  *\n                          *  Germany 4040 Neuss  *  Voice: 49 2131 82238 *\n",
 'From

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# \w - 문자+숫자(alphanumeric)와 매치, [a-zA-Z0-9_]와 동일한 표현식이다.
cv = CountVectorizer(token_pattern="[\w']{3,}", stop_words='english', 
                     max_features=2500, min_df=5, max_df=0.5)

review_cv = cv.fit_transform(newsgroups_train.data)

print(review_cv[:5])

  (0, 1399)	5
  (0, 983)	2
  (0, 678)	1
  (0, 2467)	1
  (0, 1023)	1
  (0, 1493)	1
  (0, 2272)	1
  (0, 2376)	1
  (0, 1649)	1
  (0, 1731)	1
  (0, 1623)	2
  (0, 2440)	1
  (0, 2306)	1
  (0, 2479)	1
  (0, 1737)	2
  (0, 1314)	2
  (0, 2246)	1
  (0, 2259)	1
  (0, 1840)	1
  (0, 667)	1
  (0, 2465)	1
  (0, 1182)	1
  (0, 439)	2
  (0, 2307)	1
  (0, 2122)	1
  :	:
  (3, 1405)	1
  (3, 1117)	1
  (4, 678)	1
  (4, 1023)	1
  (4, 2343)	1
  (4, 1490)	1
  (4, 2245)	1
  (4, 1501)	1
  (4, 1698)	1
  (4, 1050)	1
  (4, 456)	3
  (4, 1303)	1
  (4, 1296)	1
  (4, 1281)	1
  (4, 1885)	1
  (4, 965)	1
  (4, 2086)	1
  (4, 1906)	1
  (4, 934)	1
  (4, 2424)	1
  (4, 1142)	1
  (4, 1201)	1
  (4, 1481)	1
  (4, 2005)	1
  (4, 663)	1


### 7.2.2 LDA 토픽 모델링 실행

In [21]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np 
np.set_printoptions(precision=3)

lda = LatentDirichletAllocation(n_components= 12,   # 추출할 topic의 수 
                                max_iter = 5, 
                                topic_word_prior = 0.1, 
                                doc_topic_prior= 1.0,
                                learning_method='online',
                                n_jobs= -1,  #사용 processor 수
                                random_state=0)

review_topics = lda.fit_transform(review_cv)

print('#shape of review_topics:', review_topics.shape)
print('#sample of review_topics:', review_topics[0])

gross_topic_weights = np.mean(review_topics, axis=0)
print('#sum of topic weights of documents:', gross_topic_weights)

print('#shape of topic word distribution:', lda.components_.shape)


#shape of review_topics: (3819, 12)
#sample of review_topics: [0.032 0.039 0.492 0.022 0.021 0.02  0.019 0.084 0.149 0.021 0.06  0.041]
#sum of topic weights of documents: [0.091 0.115 0.095 0.107 0.094 0.065 0.081 0.096 0.066 0.079 0.042 0.068]
#shape of topic word distribution: (12, 2500)


In [42]:
print(review_topics[1][:])

print() 

print(lda.components_[:5])

[0.036 0.18  0.511 0.024 0.035 0.051 0.029 0.059 0.028 0.017 0.014 0.015]

[[ 0.103  0.281  0.559 ...  0.1    0.101  0.1  ]
 [ 0.1   49.408 20.554 ...  0.101  1.059  0.164]
 [ 0.1    0.119 13.108 ...  0.101  0.102  0.101]
 [ 0.1    3.819  0.111 ...  0.101  0.105  0.102]
 [58.695  7.734 36.253 ... 16.795  0.102  0.1  ]]


In [ ]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
